Runs the FIR model fits for the models defined in workflow #6 [6_define-FIR-models](http://localhost:8888/notebooks/8_define-FIR-models.ipynb).

Note: **you need to have run the function redef_FIRs.m, located on scratch, on the outputs of workflow #8, prior to running this notebook**

In [1]:
from bids.layout import BIDSLayout 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html
#
# set spm path to copy with use edit of spm_fMRI_design.m [see: https://www.jiscmail.ac.uk/cgi-bin/webadmin?A2=spm;f525092b.1508]
# scratch/qbi/uqkgarn1
matlab_cmd = '/scratch/qbi/uqkgarn1/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
layout = BIDSLayout(Basedir)
subs = layout.get_subjects()

fitfirs = pe.Workflow(name="fitfirs") # workflow to run the analysis

/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/bids/layout/models.py:102: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


# grab and sink data

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                            outfields=['spm']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'spm': [['sub','TR']]}
dg.inputs.field_template = {'spm': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-%s/TR%s/FLGLM/SPM.mat'}

In [4]:
# debugging
dg.inputs.sub = '01'
dg.inputs.TR = '1510'
res = dg.run()
res.outputs

210522-18:40:20,504 nipype.workflow INFO:
	 [Node] Setting-up "data-grabber" in "/tmp/tmpjqo9xw30/data-grabber".
210522-18:40:20,510 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210522-18:40:20,517 nipype.workflow INFO:
	 [Node] Finished "data-grabber".



spm = /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR1510/FLGLM/SPM.mat

In [5]:
inf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                   name='info')
inf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
# inf.iterables = [('sub', ['01']), ('TR', ['1510'])]
inf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0]
    TR = [t for t in l if re.search('TR([0-9]*)',t)][-1]
    name = [name, TR, "FLGLM"]
    name = '/'.join(name)
    return name

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_gzip_([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# estimate model

In [8]:
# EstimateModel - estimate the parameters of the model
estimate = pe.Node(spm.EstimateModel(estimation_method={'Classical': 1},
                                     write_residuals=False),
                                     name="estimate")

# zip files

from .nii -> .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs

In [10]:
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

# Connect workflow

In [11]:
fitfirs.connect([(inf, dg, [('sub',  'sub')]),
                 (inf, dg, [('TR', 'TR')]),
                 (dg, ds, [(('spm', printSubPath), 
                            'container')]),
                 (dg, estimate, [('spm', 'spm_mat_file')]),
                 (estimate, gzip, [('beta_images', 'in_file')]),
                 (gzip, ds, [('out_file', 'PARAMS')])
])

In [12]:
fitfirs.run()

210522-18:40:27,812 nipype.workflow INFO:
	 Workflow fitfirs settings: ['check', 'execution', 'logging', 'monitoring']
210522-18:40:27,890 nipype.workflow INFO:
	 Running serially.
210522-18:40:27,891 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.data-grabber" in "/tmp/tmpjqo9xw30/data-grabber".
210522-18:40:27,896 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210522-18:40:27,901 nipype.workflow INFO:
	 [Node] Finished "fitfirs.data-grabber".
210522-18:40:27,902 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.estimate" in "/tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate".
210522-18:40:27,933 nipype.workflow INFO:
	 [Node] Running "estimate" ("nipype.interfaces.spm.model.EstimateModel")
210522-18:47:31,820 nipype.workflow INFO:
	 [Node] Finished "fitfirs.estimate".
210522-18:47:31,823 nipype.workflow INFO:
	 [Node] Setting-up "fitfirs.gzip" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip".
210522-18:47:31,845 nipype.workflow INFO:

210522-18:47:35,506 nipype.workflow INFO:
	 [Node] Finished "_gzip17".
210522-18:47:35,508 nipype.workflow INFO:
	 [Node] Setting-up "_gzip18" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip18".
210522-18:47:35,512 nipype.workflow INFO:
	 [Node] Running "_gzip18" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0019.nii
210522-18:47:35,718 nipype.workflow INFO:
	 [Node] Finished "_gzip18".
210522-18:47:35,720 nipype.workflow INFO:
	 [Node] Setting-up "_gzip19" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip19".
210522-18:47:35,724 nipype.workflow INFO:
	 [Node] Running "_gzip19" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0020.nii
210522-18:47:35,894 nipype.workflow INFO:
	 [Node] Finished "_gzip19".
210522-18:47:35,896 nipype.workflow INFO:
	 [Node] Setting-up "_gzip20" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920

210522-18:47:39,398 nipype.workflow INFO:
	 [Node] Setting-up "_gzip38" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip38".
210522-18:47:39,401 nipype.workflow INFO:
	 [Node] Running "_gzip38" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0039.nii
210522-18:47:39,570 nipype.workflow INFO:
	 [Node] Finished "_gzip38".
210522-18:47:39,572 nipype.workflow INFO:
	 [Node] Setting-up "_gzip39" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip39".
210522-18:47:39,575 nipype.workflow INFO:
	 [Node] Running "_gzip39" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0040.nii
210522-18:47:39,777 nipype.workflow INFO:
	 [Node] Finished "_gzip39".
210522-18:47:39,779 nipype.workflow INFO:
	 [Node] Setting-up "_gzip40" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip40".
210522-18:47:39,783 nipype.workflow INFO

210522-18:47:43,379 nipype.workflow INFO:
	 [Node] Setting-up "_gzip58" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip58".
210522-18:47:43,384 nipype.workflow INFO:
	 [Node] Running "_gzip58" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0059.nii
210522-18:47:43,555 nipype.workflow INFO:
	 [Node] Finished "_gzip58".
210522-18:47:43,557 nipype.workflow INFO:
	 [Node] Setting-up "_gzip59" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip59".
210522-18:47:43,561 nipype.workflow INFO:
	 [Node] Running "_gzip59" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0060.nii
210522-18:47:43,772 nipype.workflow INFO:
	 [Node] Finished "_gzip59".
210522-18:47:43,774 nipype.workflow INFO:
	 [Node] Setting-up "_gzip60" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip60".
210522-18:47:43,779 nipype.workflow INFO

210522-18:47:47,533 nipype.workflow INFO:
	 [Node] Setting-up "_gzip78" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip78".
210522-18:47:47,537 nipype.workflow INFO:
	 [Node] Running "_gzip78" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0079.nii
210522-18:47:47,739 nipype.workflow INFO:
	 [Node] Finished "_gzip78".
210522-18:47:47,741 nipype.workflow INFO:
	 [Node] Setting-up "_gzip79" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip79".
210522-18:47:47,745 nipype.workflow INFO:
	 [Node] Running "_gzip79" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpcenc4kce/fitfirs/_TR_1920_sub_05/estimate/beta_0080.nii
210522-18:47:47,949 nipype.workflow INFO:
	 [Node] Finished "_gzip79".
210522-18:47:47,951 nipype.workflow INFO:
	 [Node] Setting-up "_gzip80" in "/tmp/tmpiqyi1sag/fitfirs/_TR_1920_sub_05/gzip/mapflow/_gzip80".
210522-18:47:47,955 nipype.workflow INFO

...

210523-05:50:39,600 nipype.workflow INFO:
	 [Node] Running "_gzip4" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0005.nii
210523-05:50:39,761 nipype.workflow INFO:
	 [Node] Finished "_gzip4".
210523-05:50:39,763 nipype.workflow INFO:
	 [Node] Setting-up "_gzip5" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip5".
210523-05:50:39,767 nipype.workflow INFO:
	 [Node] Running "_gzip5" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0006.nii
210523-05:50:39,985 nipype.workflow INFO:
	 [Node] Finished "_gzip5".
210523-05:50:39,987 nipype.workflow INFO:
	 [Node] Setting-up "_gzip6" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip6".
210523-05:50:39,992 nipype.workflow INFO:
	 [Node] Running "_gzip6" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0007

210523-05:50:43,436 nipype.workflow INFO:
	 [Node] Finished "_gzip24".
210523-05:50:43,438 nipype.workflow INFO:
	 [Node] Setting-up "_gzip25" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip25".
210523-05:50:43,443 nipype.workflow INFO:
	 [Node] Running "_gzip25" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0026.nii
210523-05:50:43,613 nipype.workflow INFO:
	 [Node] Finished "_gzip25".
210523-05:50:43,615 nipype.workflow INFO:
	 [Node] Setting-up "_gzip26" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip26".
210523-05:50:43,619 nipype.workflow INFO:
	 [Node] Running "_gzip26" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0027.nii
210523-05:50:43,770 nipype.workflow INFO:
	 [Node] Finished "_gzip26".
210523-05:50:43,772 nipype.workflow INFO:
	 [Node] Setting-up "_gzip27" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_

210523-05:50:47,215 nipype.workflow INFO:
	 [Node] Setting-up "_gzip45" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip45".
210523-05:50:47,219 nipype.workflow INFO:
	 [Node] Running "_gzip45" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0046.nii
210523-05:50:47,365 nipype.workflow INFO:
	 [Node] Finished "_gzip45".
210523-05:50:47,367 nipype.workflow INFO:
	 [Node] Setting-up "_gzip46" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip46".
210523-05:50:47,371 nipype.workflow INFO:
	 [Node] Running "_gzip46" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /tmp/tmpqsffy9yk/fitfirs/_TR_700_sub_01/estimate/beta_0047.nii
210523-05:50:47,548 nipype.workflow INFO:
	 [Node] Finished "_gzip46".
210523-05:50:47,549 nipype.workflow INFO:
	 [Node] Setting-up "_gzip47" in "/tmp/tmphbmprobw/fitfirs/_TR_700_sub_01/gzip/mapflow/_gzip47".
210523-05:50:47,554 nipype.workflow INFO:
	 [

210523-05:50:50,504 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_0005.nii.gz
210523-05:50:50,508 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_0006.nii.gz
210523-05:50:50,513 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_0007.nii.gz
210523-05:50:50,517 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0008.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_

210523-05:50:50,643 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0038.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_0038.nii.gz
210523-05:50:50,647 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0039.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_0039.nii.gz
210523-05:50:50,651 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0040.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_0040.nii.gz
210523-05:50:50,655 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS/_TR_700_sub_01/beta_0041.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmFIR/sub-01/TR700/FLGLM/PARAMS//beta_